In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
import sys
import os

# Add the parent directory (Auditbot_backend) to the system path
sys.path.append(
    os.path.abspath(
        os.path.join(
            os.path.dirname(f"{os.getcwd()}/ReRAG.ipynb"),
            '..'
        )
    )
)

from utils.langsmith_trace import chat_gpt, llm
from IPython.display import display_markdown


import tiktoken

# Load the tokenizer for the specific model
encoding = tiktoken.encoding_for_model("gpt-4")

In [24]:
# EXPERIMENT - Compare between including RAG outputs (backend) and 
#                                          LLM outputs (frontend)

# We want to use an agent to look at the conversation history and determine if 
# user's question requires RAG. This experiment is to determine if including the
# retrieved chunks for previous retrievals is useful. 

# Seems like frontend is better as shorter context needed but neccesary details
# are still included 

messages_both = [
    {'role': 'system', 'content': 'You are a helpful assistant.'}, 
    {'role': 'user', 'content': "Role:\nYou are a specialist who uses the context provided to answer the query.\n\nInstruction:\nYour response should cite sources' year and page number.\nIf possible, make ministries or government agencies the headings.\n\nBackground:\nThe context is taken from audit reports from the Auditor-General's Office (AGO) of Singapore. \nAGO is an independent organ of state and the national auditor. They play an important role in enhancing public accountability in the management and use of public funds and resources through their audits.\n\nThey audit\n    government ministries and departments\n    organs of state\n    statutory boards\n    government funds\n    other public authorities and bodies administering public funds (upon their request for audit), e.g. government-owned companies.\n\nThey report their audit observations to the President, Parliament and the public through the Annual Report of the Auditor-General management of the organisations audited through management letters.\nTheir observations include system weaknesses, non-compliance with control procedures or legislation, and instances of excess, extravagance, or gross inefficiency leading to waste in the use of public funds and resources.\n\nCONTEXT\nContext 0:\nYear: 2014_15\nLocation in document: PART I B: AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 9\nContent: In the course of the audit of the Government Financial Statements (GFS), \nAGO carries out test checks of internal controls of selected areas in Government \nministries and organs of state.  These include checks for financial irregularity, excess, \nextravagance, or gross inefficiency leading to waste in the use of funds and resources, \nand on whether measures to prevent such lapses are in place.  In this regard, AGO \nalso takes into account complaints received on the use and management of public \nfunds and resources.  The authority for these audits is provided by section 5 of the \nAudit Act\n\nContext 1:\nYear: 2020_21\nLocation in document: CONTENTS, PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 13\nContent: In the course of the audit of the Government Financial Statements (GFS),\nAGO carries out test checks of internal controls of selected areas in Government\nministries and organs of state.  These include checks for financial irregularity, excess,\nextravagance, or gross inefficiency leading to waste in the use of funds and resources,\nand on whether measures to prevent such lapses are in place.  The authority for these\naudits is provided for in section 5 of the Audit Act\n\nContext 2:\nYear: 2020_21\nLocation in document: CONTENTS, OVERVIEW, SUMMARY\nPage number: 8\nContent: Examples include the implementation of a smart mobile application for school \npersonnel to report FM-related faults and pilots on smart FM technologies such as \ncleaning robots and smart energy and water meters.  MHA had developed a competency \nframework and training plan to build up FM capabilities in the Home Team.  It had \nalso centralised the management of FM contracts across the Home Team under its new \nstatutory board, the Home Team Science and Technology Agency.  The centralised \nmodel enabled MHA to have an effective overview of all its FM contracts, and build \ndeeper FM expertise through more targeted competency and career development of \nits FM officers\n\nContext 3:\nYear: 2016_17\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 12\nContent: In the course of the audit of the Government Financial Statements (GFS), \nAGO carries out test checks of internal controls of selected areas in Government \nministries and organs of state.  These include checks for financial irregularity, excess, \nextravagance or gross inefficiency leading to waste in the use of funds and resources, \nand on whether measures to prevent such lapses are in place.  The authority for these \naudits is provided by section 5 of the Audit Act\n\nContext 4:\nYear: 2021_22\nLocation in document: PART III : THEMATIC AUDIT – COVID-19 RELATED PROCUREMENT AND EXPENDITURE, SUMMARY\nPage number: 55\nContent: Affirm and support the development of critical corporate functions \nsuch as procurement, finance and human resource\nDuring the pandemic, the focus of the public sector was on the \nfront-line and operational response.  Such front-line and operational \nfunctions should be complemented by effective corporate functions \nwhich are critical in a crisis.  This would include corporate functions \nsuch as procurement, finance and human resource.  Central agencies \nmay wish to consider how such critical support functions can be better \norganised at the WOG or cluster/agency level, and what detailed \nguidance to give agencies in these areas\n\nContext 5:\nYear: 2014_15\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF CULTURE, COMMUNITY AND YOUTH, People’s Association, Lapses in Management of Tenancy Contracts and Procurement\nPage number: 25\nContent: The number of lapses detected points to a weakness in the People’s \nAssociation (PA)’s monitoring of CCMCs’ compliance with its Financial Rules with \nregard to tenancy contracts\n\nContext 6:\nYear: 2018_19\nLocation in document: OVERVIEW, SUMMARY\nPage number: 10\nContent: Some of the lapses highlighted in this Report, such as weaknesses in IT controls and \nlapses in procurement and contract management, are similar to those reported last \nyear although the lapses involve different entities.  It is important that public sector \nentities avoid repeating similar lapses and implement effective measures to enhance \ngovernance and controls on the use of public funds\n\nContext 7:\nYear: 2019_20\nLocation in document: PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF TRADE AND INDUSTRY, Jurong Town Corporation, Unauthorised Subletting of Leased and Tenanted Premises\nPage number: 42\nContent: JTC allows lessees and tenants10 to sublet part of their premises to related or \nother entities with JTC’s approval.  When subletting, lessees or tenants are required \nto comply with conditions such as the maximum sublet area and payment of sublet \nfees11.  In financial year 2018/19, JTC collected $13.61 million in sublet fees\n\nContext 8:\nYear: 2021_22\nLocation in document: OVERVIEW, SUMMARY\nPage number: 11\nContent: Central agencies may wish to consider how critical support functions such \nas procurement, finance and human resource can be better organised at the \nWOG or cluster/agency level, and what detailed guidance to give agencies \nin these areas\n\nContext 9:\nYear: 2020_21\nLocation in document: CONTENTS, PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF NATIONAL DEVELOPMENT, Housing and Development Board, Unauthorised Subletting of Tenanted Commercial Premises\nPage number: 67\nContent: HDB allows tenants to sublet part of their premises to related or other\nentities with HDB’s approval.  When subletting, tenants are required to comply\nwith conditions such as maximum sublet area and payment of administrative fee\n\nContext 10:\nYear: 2019_20\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, PRIME MINISTER’S OFFICE, Public Service Division, Weak Controls over Privileged Operating System Accounts\nPage number: 17\nContent: The Public Service Division (PSD) uses the Human Resource Management \nSystem (HRMS) to administer personnel information (e.g. personal particulars, \ntraining and leave records) and payroll-related information of civil servants.  The \nHRMS is also used by civil servants for HR-related activities such as applying for \nleave and filling in staff appraisal forms\n\nContext 11:\nYear: 2016_17\nLocation in document: OVERVIEW, SUMMARY\nPage number: 2\nContent: In selecting areas for audit, one of the key factors AGO considers is the materiality of \ntransactions.  Dollar value is an important consideration in determining materiality \nbut it is not the only consideration.  AGO also considers other factors such as the \npotential impact an irregularity in a particular area may have on the entity or the \npublic sector as a whole\n\nContext 12:\nYear: 2012_13\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF EDUCATION, Republic Polytechnic, Related Party Given Preferential Treatment\nPage number: 41\nContent: For the provision of travel-related services such as airfares and hotel \naccommodation, RP’s practice was to appoint a panel of vendors based on the \nmerits (e.g. area of expertise) of each vendor in meeting RP’s requirements\n\nContext 13:\nYear: 2019_20\nLocation in document: PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF COMMUNICATIONS AND INFORMATION, National Library Board, Lapses in Management of Contract Variations for the National Archives Development Project\nPage number: 31\nContent: AGO also noted that there was inadequate monitoring of project expenditure \nto ensure that financial commitments were kept within the approved project cost.  The \nproject eventually exceeded the approved project cost by $1.72 million, which was \nabout 8.4 per cent of the approved project cost of $20.53 million.  In addition, NLB \nonly sought approval from its approving authority for exceeding the approved project \ncost five months after it became aware of the cost overrun.  Seeking retrospective \napproval undermined the role of the approving authority and indicated a weakness \nin financial controls\n\nContext 14:\nYear: 2013_14\nLocation in document: OVERVIEW, SUMMARY\nPage number: 5\nContent: Management of Land and Assets\nPublic sector entities may be assigned land for specific uses.  As land is a valuable \nresource, these entities are expected to manage such land in accordance with the \nlaw and Government policy objectives.  For proper management and regulation of \nland resources, it is important that all uses of land are properly covered by licence or \nlease agreements to protect the rights of the various parties, should any dispute arise\n\nContext 15:\nYear: 2014_15\nLocation in document: OVERVIEW, SUMMARY\nPage number: 2\nContent: AGO adopts a risk-based approach in determining the areas to be covered in an \naudit.  In selecting areas for audit, AGO considers factors such as financial value of \ntransactions and the potential impact an irregularity in a particular area may have \non the organisation or the public sector as a whole\n\nContext 16:\nYear: 2021_22\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, MINISTRY OF DEFENCE, Errors in Payment of Flexible Benefits Allowances\nPage number: 18\nContent: AGO recommended that MINDEF strengthen its review processes before the \nimplementation of any system changes.  As for the errors in paragraph 9b, these were \nnot detected as checks over entry of personnel data such as date of re-employment \ninto the HR system were not consistently and promptly performed.  It is important to \nperform accuracy checks on the entry of personnel data into the HR system as such \ninformation would be used in the computation of various benefits and allowances\n\nContext 17:\nYear: 2021_22\nLocation in document: OVERVIEW, SUMMARY\nPage number: 7\nContent: To help agencies with their COVID-19 related procurement, the Ministry of Finance \nimplemented a number of measures, such as the central activation of Emergency \nProcurement (EP) procedures at the WOG level in late January 2020.  All agencies \nwere also required to submit an Accountability Report, documenting all EP decisions, \nto their Permanent Secretaries for endorsement\n\nContext 18:\nYear: 2014_15\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF COMMUNICATIONS AND INFORMATION, Info-communications Development Authority of Singapore, Lapses in Administration of Project Funding\nPage number: 21\nContent: IDA administers funding of projects pertaining to Information and \nCommunications Technology development.  These projects could be funded by \ngrants from ministries or IDA depending on factors such as the nature of the project\n\nContext 19:\nYear: 2018_19\nLocation in document: OVERVIEW, SUMMARY\nPage number: 5\nContent: 5\nOverview\n(1)\t\nWeaknesses in IT Controls\nThe public sector is leveraging on technology in many ways such as digitalisation \nand automation of processes to better serve individuals and businesses.  With the \nvast amounts of data managed, which includes personal and confidential data, any \nunauthorised access or activity could have significant implication on the integrity \nand confidentiality of the data in the IT systems.  It is therefore critical that the public \nsector ensures that the IT controls put in place are sufficiently robust to prevent and \ndetect unauthorised access and activities\n\nQuery: what are the findings pertaining to weakness?"},
    {'role': 'assistant', 'content': "### Government Ministries and Departments\n\n**General Weaknesses**\nIn the audit of Government Financial Statements (GFS) for multiple years, the AGO identified recurring weaknesses in internal controls across various government ministries and organs of state. These checks included assessments for financial irregularity, excess, extravagance, and gross inefficiency, which potentially lead to wasteful use of funds and resources (e.g., 2014-15, p. 9; 2016-17, p. 12; 2020-21, p. 13).\n\n### Ministry of Culture, Community, and Youth\n\n**People’s Association: Financial Rule Compliance**\nThe AGO found that the People’s Association had notable lapses in the management of tenancy contracts and procurement. These lapses indicated a systemic weakness in monitoring compliance with its Financial Rules by Constituency Management Committees (CCMCs) (2014-15, p. 25).\n\n### Ministry of Trade and Industry\n\n**Jurong Town Corporation: Unauthorised Subletting**\nThe Jurong Town Corporation was scrutinized for allowing lessees and tenants to sublet premises with inadequate checks for compliance. This involved unauthorized subletting activities, which indicated weaknesses in the subletting management system (2019-20, p. 42).\n\n### Ministry of National Development\n\n**Housing and Development Board: Unauthorised Subletting**\nSimilar to the case with the Jurong Town Corporation, the Housing and Development Board was found to have inadequate controls over the subletting of tenanted commercial premises, indicating a systemic issue in managing subletting agreements and compliance requirements (2020-21, p. 67).\n\n### Ministry of Communications and Information\n\n**National Library Board: Project Cost Management**\nThe National Library Board had significant lapses in managing contract variations for the National Archives Development Project. The project exceeded the approved cost by $1.72 million without timely approval, highlighting weaknesses in financial controls and project expenditure monitoring (2019-20, p. 31).\n\n### Cross-Agency Weaknesses\n\n**IT Control Weaknesses**\nMultiple audit reports across different years have pointed to weaknesses in IT controls within the public sector. These lapses could lead to unauthorized access to confidential data, thereby compromising the data’s integrity and confidentiality (2018-19, pp. 5, 10).\n\n### Specific Findings\n\n**Public Service Division: Privileged OS Accounts**\nThe AGO noted weak controls over privileged operating system accounts used within the Public Service Division’s HR Management System. These weaknesses posed risks to the system’s security and data integrity (2019-20, p. 17).\n\n**Ministry of Defence: Flexible Benefits Allowance**\nThe Ministry of Defence displayed errors in the payment of flexible benefits allowances due to inconsistent checks on entry of personnel data into the HR system. The AGO recommended stronger review processes before implementing any system changes to mitigate such errors (2021-22, p. 18).\n\n**Environmental Management**\nThe Ministry of Home Affairs (MHA) centralized Facilities Management (FM) contracts across its units through the Home Team Science and Technology Agency to ensure better oversight and deeper expertise. However, recurring lapses in contract management suggest a broader challenge in implementing effective contract and resource management practices (2020-21, p. 8).\n\n### Recommendations and Observations\n\n**Central Coordination for Critical Functions**\nThe AGO suggested that central agencies consider better organizing critical support functions such as procurement, finance, and human resources at whole-of-government or specific cluster/agency levels. This recommendation emphasizes the necessity of robust corporate functions to support operational activities, especially during crises like the COVID-19 pandemic (2021-22, pp. 11, 55; 2020-21, p. 8).\n\n### Concluding Note\nThe Auditor-General's Office has repeatedly identified weaknesses in various government sectors, emphasizing the need for improved internal controls, better IT management, and stricter compliance with financial regulations and project management protocols. The recurring nature of some of these issues highlights ongoing challenges in governance and public sector administration.\n\nSources:\n- 2014-15, p. 9, 21, 25\n- 2016-17, p. 12\n- 2018-19, pp. 5, 10\n- 2019-20, p. 17, 31, 42\n- 2020-21, pp. 8, 13, 67\n- 2021-22, pp. 11, 18, 55"}
]

messages_frontend = [
    {'role': 'system', 'content': 'You are a helpful assistant.'}, 
    {'role': 'user', 'content': 'what are the findings pertaining to weakness?'}, 
    {'role': 'assistant', 'content': "### Government Ministries and Departments\n\n**General Weaknesses**\nIn the audit of Government Financial Statements (GFS) for multiple years, the AGO identified recurring weaknesses in internal controls across various government ministries and organs of state. These checks included assessments for financial irregularity, excess, extravagance, and gross inefficiency, which potentially lead to wasteful use of funds and resources (e.g., 2014-15, p. 9; 2016-17, p. 12; 2020-21, p. 13).\n\n### Ministry of Culture, Community, and Youth\n\n**People’s Association: Financial Rule Compliance**\nThe AGO found that the People’s Association had notable lapses in the management of tenancy contracts and procurement. These lapses indicated a systemic weakness in monitoring compliance with its Financial Rules by Constituency Management Committees (CCMCs) (2014-15, p. 25).\n\n### Ministry of Trade and Industry\n\n**Jurong Town Corporation: Unauthorised Subletting**\nThe Jurong Town Corporation was scrutinized for allowing lessees and tenants to sublet premises with inadequate checks for compliance. This involved unauthorized subletting activities, which indicated weaknesses in the subletting management system (2019-20, p. 42).\n\n### Ministry of National Development\n\n**Housing and Development Board: Unauthorised Subletting**\nSimilar to the case with the Jurong Town Corporation, the Housing and Development Board was found to have inadequate controls over the subletting of tenanted commercial premises, indicating a systemic issue in managing subletting agreements and compliance requirements (2020-21, p. 67).\n\n### Ministry of Communications and Information\n\n**National Library Board: Project Cost Management**\nThe National Library Board had significant lapses in managing contract variations for the National Archives Development Project. The project exceeded the approved cost by $1.72 million without timely approval, highlighting weaknesses in financial controls and project expenditure monitoring (2019-20, p. 31).\n\n### Cross-Agency Weaknesses\n\n**IT Control Weaknesses**\nMultiple audit reports across different years have pointed to weaknesses in IT controls within the public sector. These lapses could lead to unauthorized access to confidential data, thereby compromising the data’s integrity and confidentiality (2018-19, pp. 5, 10).\n\n### Specific Findings\n\n**Public Service Division: Privileged OS Accounts**\nThe AGO noted weak controls over privileged operating system accounts used within the Public Service Division’s HR Management System. These weaknesses posed risks to the system’s security and data integrity (2019-20, p. 17).\n\n**Ministry of Defence: Flexible Benefits Allowance**\nThe Ministry of Defence displayed errors in the payment of flexible benefits allowances due to inconsistent checks on entry of personnel data into the HR system. The AGO recommended stronger review processes before implementing any system changes to mitigate such errors (2021-22, p. 18).\n\n**Environmental Management**\nThe Ministry of Home Affairs (MHA) centralized Facilities Management (FM) contracts across its units through the Home Team Science and Technology Agency to ensure better oversight and deeper expertise. However, recurring lapses in contract management suggest a broader challenge in implementing effective contract and resource management practices (2020-21, p. 8).\n\n### Recommendations and Observations\n\n**Central Coordination for Critical Functions**\nThe AGO suggested that central agencies consider better organizing critical support functions such as procurement, finance, and human resources at whole-of-government or specific cluster/agency levels. This recommendation emphasizes the necessity of robust corporate functions to support operational activities, especially during crises like the COVID-19 pandemic (2021-22, pp. 11, 55; 2020-21, p. 8).\n\n### Concluding Note\nThe Auditor-General's Office has repeatedly identified weaknesses in various government sectors, emphasizing the need for improved internal controls, better IT management, and stricter compliance with financial regulations and project management protocols. The recurring nature of some of these issues highlights ongoing challenges in governance and public sector administration.\n\nSources:\n- 2014-15, p. 9, 21, 25\n- 2016-17, p. 12\n- 2018-19, pp. 5, 10\n- 2019-20, p. 17, 31, 42\n- 2020-21, pp. 8, 13, 67\n- 2021-22, pp. 11, 18, 55"}
]

messages_backend = [
    {'role': 'system', 'content': 'You are a helpful assistant.'}, 
    {'role': 'user', 'content': "Role:\nYou are a specialist who uses the context provided to answer the query.\n\nInstruction:\nYour response should cite sources' year and page number.\nIf possible, make ministries or government agencies the headings.\n\nBackground:\nThe context is taken from audit reports from the Auditor-General's Office (AGO) of Singapore. \nAGO is an independent organ of state and the national auditor. They play an important role in enhancing public accountability in the management and use of public funds and resources through their audits.\n\nThey audit\n    government ministries and departments\n    organs of state\n    statutory boards\n    government funds\n    other public authorities and bodies administering public funds (upon their request for audit), e.g. government-owned companies.\n\nThey report their audit observations to the President, Parliament and the public through the Annual Report of the Auditor-General management of the organisations audited through management letters.\nTheir observations include system weaknesses, non-compliance with control procedures or legislation, and instances of excess, extravagance, or gross inefficiency leading to waste in the use of public funds and resources.\n\nCONTEXT\nContext 0:\nYear: 2014_15\nLocation in document: PART I B: AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 9\nContent: In the course of the audit of the Government Financial Statements (GFS), \nAGO carries out test checks of internal controls of selected areas in Government \nministries and organs of state.  These include checks for financial irregularity, excess, \nextravagance, or gross inefficiency leading to waste in the use of funds and resources, \nand on whether measures to prevent such lapses are in place.  In this regard, AGO \nalso takes into account complaints received on the use and management of public \nfunds and resources.  The authority for these audits is provided by section 5 of the \nAudit Act\n\nContext 1:\nYear: 2020_21\nLocation in document: CONTENTS, PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 13\nContent: In the course of the audit of the Government Financial Statements (GFS),\nAGO carries out test checks of internal controls of selected areas in Government\nministries and organs of state.  These include checks for financial irregularity, excess,\nextravagance, or gross inefficiency leading to waste in the use of funds and resources,\nand on whether measures to prevent such lapses are in place.  The authority for these\naudits is provided for in section 5 of the Audit Act\n\nContext 2:\nYear: 2020_21\nLocation in document: CONTENTS, OVERVIEW, SUMMARY\nPage number: 8\nContent: Examples include the implementation of a smart mobile application for school \npersonnel to report FM-related faults and pilots on smart FM technologies such as \ncleaning robots and smart energy and water meters.  MHA had developed a competency \nframework and training plan to build up FM capabilities in the Home Team.  It had \nalso centralised the management of FM contracts across the Home Team under its new \nstatutory board, the Home Team Science and Technology Agency.  The centralised \nmodel enabled MHA to have an effective overview of all its FM contracts, and build \ndeeper FM expertise through more targeted competency and career development of \nits FM officers\n\nContext 3:\nYear: 2016_17\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, SUMMARY\nPage number: 12\nContent: In the course of the audit of the Government Financial Statements (GFS), \nAGO carries out test checks of internal controls of selected areas in Government \nministries and organs of state.  These include checks for financial irregularity, excess, \nextravagance or gross inefficiency leading to waste in the use of funds and resources, \nand on whether measures to prevent such lapses are in place.  The authority for these \naudits is provided by section 5 of the Audit Act\n\nContext 4:\nYear: 2021_22\nLocation in document: PART III : THEMATIC AUDIT – COVID-19 RELATED PROCUREMENT AND EXPENDITURE, SUMMARY\nPage number: 55\nContent: Affirm and support the development of critical corporate functions \nsuch as procurement, finance and human resource\nDuring the pandemic, the focus of the public sector was on the \nfront-line and operational response.  Such front-line and operational \nfunctions should be complemented by effective corporate functions \nwhich are critical in a crisis.  This would include corporate functions \nsuch as procurement, finance and human resource.  Central agencies \nmay wish to consider how such critical support functions can be better \norganised at the WOG or cluster/agency level, and what detailed \nguidance to give agencies in these areas\n\nContext 5:\nYear: 2014_15\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF CULTURE, COMMUNITY AND YOUTH, People’s Association, Lapses in Management of Tenancy Contracts and Procurement\nPage number: 25\nContent: The number of lapses detected points to a weakness in the People’s \nAssociation (PA)’s monitoring of CCMCs’ compliance with its Financial Rules with \nregard to tenancy contracts\n\nContext 6:\nYear: 2018_19\nLocation in document: OVERVIEW, SUMMARY\nPage number: 10\nContent: Some of the lapses highlighted in this Report, such as weaknesses in IT controls and \nlapses in procurement and contract management, are similar to those reported last \nyear although the lapses involve different entities.  It is important that public sector \nentities avoid repeating similar lapses and implement effective measures to enhance \ngovernance and controls on the use of public funds\n\nContext 7:\nYear: 2019_20\nLocation in document: PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF TRADE AND INDUSTRY, Jurong Town Corporation, Unauthorised Subletting of Leased and Tenanted Premises\nPage number: 42\nContent: JTC allows lessees and tenants10 to sublet part of their premises to related or \nother entities with JTC’s approval.  When subletting, lessees or tenants are required \nto comply with conditions such as the maximum sublet area and payment of sublet \nfees11.  In financial year 2018/19, JTC collected $13.61 million in sublet fees\n\nContext 8:\nYear: 2021_22\nLocation in document: OVERVIEW, SUMMARY\nPage number: 11\nContent: Central agencies may wish to consider how critical support functions such \nas procurement, finance and human resource can be better organised at the \nWOG or cluster/agency level, and what detailed guidance to give agencies \nin these areas\n\nContext 9:\nYear: 2020_21\nLocation in document: CONTENTS, PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF NATIONAL DEVELOPMENT, Housing and Development Board, Unauthorised Subletting of Tenanted Commercial Premises\nPage number: 67\nContent: HDB allows tenants to sublet part of their premises to related or other\nentities with HDB’s approval.  When subletting, tenants are required to comply\nwith conditions such as maximum sublet area and payment of administrative fee\n\nContext 10:\nYear: 2019_20\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, PRIME MINISTER’S OFFICE, Public Service Division, Weak Controls over Privileged Operating System Accounts\nPage number: 17\nContent: The Public Service Division (PSD) uses the Human Resource Management \nSystem (HRMS) to administer personnel information (e.g. personal particulars, \ntraining and leave records) and payroll-related information of civil servants.  The \nHRMS is also used by civil servants for HR-related activities such as applying for \nleave and filling in staff appraisal forms\n\nContext 11:\nYear: 2016_17\nLocation in document: OVERVIEW, SUMMARY\nPage number: 2\nContent: In selecting areas for audit, one of the key factors AGO considers is the materiality of \ntransactions.  Dollar value is an important consideration in determining materiality \nbut it is not the only consideration.  AGO also considers other factors such as the \npotential impact an irregularity in a particular area may have on the entity or the \npublic sector as a whole\n\nContext 12:\nYear: 2012_13\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF EDUCATION, Republic Polytechnic, Related Party Given Preferential Treatment\nPage number: 41\nContent: For the provision of travel-related services such as airfares and hotel \naccommodation, RP’s practice was to appoint a panel of vendors based on the \nmerits (e.g. area of expertise) of each vendor in meeting RP’s requirements\n\nContext 13:\nYear: 2019_20\nLocation in document: PART II : AUDIT OF STATUTORY BOARDS, MINISTRY OF COMMUNICATIONS AND INFORMATION, National Library Board, Lapses in Management of Contract Variations for the National Archives Development Project\nPage number: 31\nContent: AGO also noted that there was inadequate monitoring of project expenditure \nto ensure that financial commitments were kept within the approved project cost.  The \nproject eventually exceeded the approved project cost by $1.72 million, which was \nabout 8.4 per cent of the approved project cost of $20.53 million.  In addition, NLB \nonly sought approval from its approving authority for exceeding the approved project \ncost five months after it became aware of the cost overrun.  Seeking retrospective \napproval undermined the role of the approving authority and indicated a weakness \nin financial controls\n\nContext 14:\nYear: 2013_14\nLocation in document: OVERVIEW, SUMMARY\nPage number: 5\nContent: Management of Land and Assets\nPublic sector entities may be assigned land for specific uses.  As land is a valuable \nresource, these entities are expected to manage such land in accordance with the \nlaw and Government policy objectives.  For proper management and regulation of \nland resources, it is important that all uses of land are properly covered by licence or \nlease agreements to protect the rights of the various parties, should any dispute arise\n\nContext 15:\nYear: 2014_15\nLocation in document: OVERVIEW, SUMMARY\nPage number: 2\nContent: AGO adopts a risk-based approach in determining the areas to be covered in an \naudit.  In selecting areas for audit, AGO considers factors such as financial value of \ntransactions and the potential impact an irregularity in a particular area may have \non the organisation or the public sector as a whole\n\nContext 16:\nYear: 2021_22\nLocation in document: PART I B : AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, MINISTRY OF DEFENCE, Errors in Payment of Flexible Benefits Allowances\nPage number: 18\nContent: AGO recommended that MINDEF strengthen its review processes before the \nimplementation of any system changes.  As for the errors in paragraph 9b, these were \nnot detected as checks over entry of personnel data such as date of re-employment \ninto the HR system were not consistently and promptly performed.  It is important to \nperform accuracy checks on the entry of personnel data into the HR system as such \ninformation would be used in the computation of various benefits and allowances\n\nContext 17:\nYear: 2021_22\nLocation in document: OVERVIEW, SUMMARY\nPage number: 7\nContent: To help agencies with their COVID-19 related procurement, the Ministry of Finance \nimplemented a number of measures, such as the central activation of Emergency \nProcurement (EP) procedures at the WOG level in late January 2020.  All agencies \nwere also required to submit an Accountability Report, documenting all EP decisions, \nto their Permanent Secretaries for endorsement\n\nContext 18:\nYear: 2014_15\nLocation in document: PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF COMMUNICATIONS AND INFORMATION, Info-communications Development Authority of Singapore, Lapses in Administration of Project Funding\nPage number: 21\nContent: IDA administers funding of projects pertaining to Information and \nCommunications Technology development.  These projects could be funded by \ngrants from ministries or IDA depending on factors such as the nature of the project\n\nContext 19:\nYear: 2018_19\nLocation in document: OVERVIEW, SUMMARY\nPage number: 5\nContent: 5\nOverview\n(1)\t\nWeaknesses in IT Controls\nThe public sector is leveraging on technology in many ways such as digitalisation \nand automation of processes to better serve individuals and businesses.  With the \nvast amounts of data managed, which includes personal and confidential data, any \nunauthorised access or activity could have significant implication on the integrity \nand confidentiality of the data in the IT systems.  It is therefore critical that the public \nsector ensures that the IT controls put in place are sufficiently robust to prevent and \ndetect unauthorised access and activities\n\nQuery: what are the findings pertaining to weakness?"}
]

question = "What are the weaknesses specifically regarding Ministry of Defence? Can you elaborate?"

json_object = {"answer": "YES or NO"}

prompt = f'''
Using the conversation history, if you do not have enough information to respond to the following message, then reply with a "NO", otherwise, reply with a "YES".

message: {question}
 
Answer by providing a json object : ```{json_object}``` and nothing else.
'''

agent_messages_frontend = messages_frontend + [{'role': 'user', 'content': prompt}]
agent_messages_backend = messages_backend + [{'role': 'user', 'content': prompt}]
agent_messages_both = messages_both + [{'role': 'user', 'content': prompt}]

In [25]:
response = chat_gpt(agent_messages_frontend, llm)
print("frontend: ", end=" ")
display_markdown(response, raw=True)
print()

print("backend: ", end=" ")
response = chat_gpt(agent_messages_backend, llm)
display_markdown(response, raw=True)
print()

print("both: ", end=" ")
response = chat_gpt(agent_messages_both, llm)
display_markdown(response, raw=True)
print()

frontend:  

```{'answer': 'YES'}```


backend:  

```{'answer': 'YES'}```


both:  

```{'answer': 'YES'}```

In [30]:
question = "Is there anything common between these 2 years regarding grants?"
greeting = "Hi"

json_object = {"answer": "YES or NO", "reason": "reasoning for answer given"}

messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': 'Hi, what are the findings pertaining to grant from 2011'}, 
    {'role': 'assistant', 'content': '''### Ministry of Education 
Inadequate Controls to Ensure Proper Use of Grants (2014-15):
AGO identified that a significant portion of grants allocated to schools went unused. Specifically, in the period from January 2011 to August 2013, a total of $232,900 (or 53.8 per cent) of the AEM grants allocated to eight schools was not utilized. Two schools, in particular, did not use any of the grants allocated (2014-15, p. 11).

### Economic Development Board
Lapses in Administration of Grants (2016-17):
While this observation extends beyond 2011, it's worth noting. The Economic Development Board (EDB) was found to have specific controls to ensure accurate reporting by grant recipients. These included sample checks with onsite visits by its Internal Audit and site visits by its Cluster Groups, which applied to five of the nine schemes audited by AGO (2016-17, p. 48).

### Workforce Singapore Agency
Inconsistent Practice in Determining Grant Quantum (2019-20):
Though the year here is much later than 2011, this finding highlights the need for greater oversight in grant determinations. One project received significantly less funding due to inconsistent practices in grant calculation (2019-20, p. 64).

### Singapore Sports Council
Audit Shared Services Scheme (2011-12):
Following a complaint, AGO audited the "request for proposal" (RFP) by the Singapore Sports Council (SSC) for its Audit Shared Services Scheme during the period from January 2010 to March 2012. The initial contract period was set from January 1, 2010, to March 31, 2011, with an extension option of 12 months (2011-12, p. 7).

### National Research Foundation and A*STAR
R&D Grant Managed by A*STAR and NRF:
From financial years 2011/12 to 2015/16, the five-year R&D grant budget managed by A*STAR and NRF totaled $5.26 billion and $3.65 billion respectively (2017-18, p. 8).

These examples highlight the findings related to grant administration and utilization, particularly from the year 2011 as observed by the Auditor-General's Office.'''},
{'role': 'user', 'content': 'What about 2018?'}, 
{'role': 'assistant', 'content': '''### Ministry of Health

**Medical Endowment Fund - Medifund Assistance Not Computed in Accordance with Medifund Manual**

In 2018, the AGO found discrepancies in the computation of Medifund assistance at a restructured hospital (RH). Specifically, the RH did not adhere to the Medifund manual guidelines for recipients who chose to stay in class B2 wards. This resulted in higher assistance than what was provided for in the manual. For example, 24 bills for 22 recipients resulted in a financial disparity amounting to $119,100 during the period of 1 April 2017 to 31 March 2018. This miscalculation had been occurring since 2009, leading MOH to estimate that approximately 4,500 class B2 ward bills might be affected from 2013 to 2018 (2018_19, p. 31).

**Source:** Auditor-General's Office, Singapore. (2018_19). "PART I B: AUDIT OF GOVERNMENT MINISTRIES, ORGANS OF STATE AND GOVERNMENT FUNDS, MINISTRY OF HEALTH", p. 31.

### Ministry of Culture, Community and Youth

**Weaknesses in Management of Access Controls of IT Systems**

The AGO’s audit in 2018 revealed that four active accounts that were no longer required were not deactivated. Two of these accounts belonged to officers who had resigned in December 2017, approximately a year before the audit. This lapse presents potential security risks and demonstrates a need for more stringent access control measures (2018_19, p. 42).

**Source:** Auditor-General's Office, Singapore. (2018_19). "PART II: AUDIT OF STATUTORY BOARDS, MINISTRY OF CULTURE, COMMUNITY AND YOUTH", p. 42.'''},
{'role': 'user', 'content': f'''
    Using the conversation history, if you do not have enough information to respond to the following message, then reply with a "NO", otherwise, reply with a "YES".
    Provide a short reason for your answer.

    message: {greeting}
 
    Answer by providing a json object : ```{json_object}``` and nothing else.
    '''}    
]


In [5]:
# code to calculate number of tokens: my 6 way conversation only amounts to 947
# tokens while GPT 4-o can take 128k tokens

text = ''
for message in messages:
    text += message['content']

# Encode the text to get the tokens
tokens = encoding.encode(text)

# Get the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")

Number of tokens: 947


In [31]:
response = chat_gpt(messages, llm)
display_markdown(response, raw=True)

```{'answer': 'YES', 'reason': 'There is sufficient interaction in the conversation history, including a greeting.'}```